In [1]:
import psycopg2
from contextlib import closing
def prettify_result(list_of_res):
    #print(list_of_res)
    try:
        return [el.strip() if type(el)==str else el for el in list_of_res]
    except:
        print("Sory,its empty")

In [2]:
with closing(psycopg2.connect(dbname="hotels",user="postgres", password="manh020598manh")) as conn:
    with conn.cursor() as cursor:
        cursor.execute('select * from hotel_rooms')
        records = cursor.fetchall()
        for el in records:
            print(prettify_result(el))

[3, None, 'Konoha', datetime.date(2020, 1, 15), datetime.date(2020, 4, 27)]
[1, 'Maxim', 'GrandPri', datetime.date(2020, 1, 5), datetime.date(2020, 5, 30)]
[1, 'Maxim', 'Raftel', datetime.date(2020, 1, 4), datetime.date(2020, 5, 15)]


In [3]:
with closing(psycopg2.connect(dbname="Tickets",user="postgres", password="manh020598manh")) as conn:
    with conn.cursor() as cursor:
        cursor.execute('select * from tickets_table')
        records = cursor.fetchall()
        for el in records:
            print(prettify_result(el))

[3, None, datetime.date(2020, 4, 15), 'KLM', 'UA', 'USA']
[2, None, datetime.date(2020, 5, 6), 'MAY', 'UK', 'UA']
[1, 'Maxim', datetime.date(2020, 5, 14), 'US', 'USA', 'USA']


In [4]:
tickets_here = False
departure_date = '2020-05-14'
with closing(psycopg2.connect(dbname="Tickets",user="postgres", password="manh020598manh")) as conn:
    with conn.cursor() as cursor:
        cursor.execute('select * from tickets_table')
        cursor.execute("update tickets_table set client_name = 'Maxim' where id = (select id from tickets_table where client_name is not NULL and departure_date='{}' limit 1)".format(departure_date))
        print(cursor.rowcount)
        if cursor.rowcount:
            cursor.execute("prepare transaction 'Maxim_update_ticket'")
            tickets_here = True

1


In [5]:
hotel_here = False
with closing(psycopg2.connect(dbname="hotels",user="postgres", password="manh020598manh")) as conn:
    with conn.cursor() as cursor:
        cursor.execute("update hotel_rooms set client_name = 'Maxim' where id = (select id from hotel_rooms where client_name is not NULL and arrival_date < '{}' and departure_date > '{}' limit 1)".format(departure_date,departure_date))
        print(cursor.rowcount)
        if cursor.rowcount:
            cursor.execute("prepare transaction 'Maxim_update_room'")
            hotel_here = True

2


In [6]:
if hotel_here and tickets_here:
    with closing(psycopg2.connect(dbname="Tickets",user="postgres", password="manh020598manh")) as conn:
        with conn.cursor() as cursor:
            conn.autocommit = True
            cursor.execute("commit prepared 'Maxim_update_ticket'")
            print("COMMITED")
            
    with closing(psycopg2.connect(dbname="hotels",user="postgres", password="manh020598manh")) as conn:
        with conn.cursor() as cursor:
            conn.autocommit = True
            cursor.execute("commit prepared 'Maxim_update_room'")
            print("COMMITED")
else:
    with closing(psycopg2.connect(dbname="Tickets",user="postgres", password="manh020598manh")) as conn:
        with conn.cursor() as cursor:
            conn.autocommit = True
            cursor.execute("rollback prepared 'Maxim_update_ticket'")
            print("ROLLBACK")
            
    with closing(psycopg2.connect(dbname="hotels",user="postgres", password="manh020598manh")) as conn:
        with conn.cursor() as cursor:
            conn.autocommit = True
            cursor.execute("rollback prepared 'Maxim_update_room'")
            print("ROLLBACK")


COMMITED
COMMITED
